# Prediction and Submission

This notebook loads a previously trained model, the test dataset and saves predictions compliant with the competition submission format.

In [1]:
import os
from pathlib import Path

from biomasstry.datasets import TemporalSentinel2Dataset
from biomasstry.models import TemporalSentinel2Model
from PIL import Image
import torch
from tqdm.notebook import tqdm

In [2]:
ARTIFACTS_DIR = Path("/notebooks/artifacts")
PREDICTIONS_DIR = Path("/notebooks/submission")
model_file = "20230102_TemporalS2_B16_E10.pt"
model_path = ARTIFACTS_DIR / model_file

## Test Data

In [3]:
# Testing Dataset
testds = TemporalSentinel2Dataset(train=False)

## Pre-trained Model

In [4]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [5]:
print(f"Device: {device}")

Device: cuda


In [6]:
model = TemporalSentinel2Model(n_samples=5, output_nc=1)
model.load_state_dict(torch.load(model_path))
model.to(device)
model.eval()

TemporalSentinel2Model(
  (model_initial): StackedResnet2D(
    (model): Sequential(
      (0): Conv2d(10, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): ResnetBlock2D(
        (conv_block): Sequential(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): ReLU(inplace=True)
          (2): Dropout(p=0.2, inplace=False)
          (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
      )
      (3): ResnetBlock2D(
        (conv_block): Sequential(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): ReLU(inplace=True)
          (2): Dropout(p=0.2, inplace=False)
          (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
      )
      (4): ResnetBlock2D(
        (conv_block): Sequential(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): ReLU(inplace=T

In [29]:
def predict_agbm(inputs, model):
    pred = model(inputs)
    return pred.detach().squeeze().cpu().numpy()

In [8]:
def save_agbm(agbm_pred, chipid):
    im = Image.fromarray(agbm_pred)
    save_path = os.path.join(PREDICTIONS_DIR, f'{chipid}_agbm.tif')
    im.save(save_path, format='TIFF', save_all=True)

In [31]:
for ix, timg in tqdm(enumerate(testds), total=len(testds)):
    chipid = testds.chip_ids[ix]
    inputs = [img.unsqueeze(0).to(device) for img in timg['image']]
    agbm = predict_agbm(inputs, model)
    save_agbm(agbm, chipid)

  0%|          | 0/2773 [00:00<?, ?it/s]